In [ ]:
# install python libraries
!pip install pyLDAvis
!pip install -U gensim

In [74]:
# import required libraries

import pandas as pd
import os
import re
import string
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

In [5]:
# read article data
data = pd.read_csv('/content/drive/MyDrive/Fiverr/govardhan/Articles.csv',encoding='latin1',error_bad_lines=False)

In [7]:
data.head()

Article      Date  \
0  KARACHI: The Sindh government has decided to b...  1/1/2015   
1  HONG KONG: Asian markets started 2015 on an up...  1/2/2015   
2  HONG KONG:  Hong Kong shares opened 0.66 perce...  1/5/2015   
3  HONG KONG: Asian markets tumbled Tuesday follo...  1/6/2015   
4  NEW YORK: US oil prices Monday slipped below $...  1/6/2015   

                                             Heading  NewsType  
0  sindh govt decides to cut public transport far...  business  
1                    asia stocks up in new year trad  business  
2           hong kong stocks open 0.66 percent lower  business  
3             asian stocks sink euro near nine year   business  
4                 us oil prices slip below 50 a barr  business

In [8]:
data.shape

(2692, 4)

In [9]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, 
    remove punctuation, remove read errors,
    and remove words containing numbers.'''
    text_l = text.split(':')
    if len(text_l)>1:
      text = text_l[1]
    text = text.lower()
    text = text.strip()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('�', ' ', text)
    return text


# Clean Article Text
data["clean_text"] = data["Article"].apply(lambda x: clean_text(x))

In [10]:
data.head()

Article      Date  \
0  KARACHI: The Sindh government has decided to b...  1/1/2015   
1  HONG KONG: Asian markets started 2015 on an up...  1/2/2015   
2  HONG KONG:  Hong Kong shares opened 0.66 perce...  1/5/2015   
3  HONG KONG: Asian markets tumbled Tuesday follo...  1/6/2015   
4  NEW YORK: US oil prices Monday slipped below $...  1/6/2015   

                                             Heading  NewsType  \
0  sindh govt decides to cut public transport far...  business   
1                    asia stocks up in new year trad  business   
2           hong kong stocks open 0.66 percent lower  business   
3             asian stocks sink euro near nine year   business   
4                 us oil prices slip below 50 a barr  business   

                                          clean_text  
0  the sindh government has decided to bring down...  
1  asian markets started   on an upswing in limit...  
2  hong kong shares opened     percent lower mond...  
3  asian markets tumbled tuesday following painfu...  
4  us oil prices monday slipped below    a barrel...

###  Models

1.   NMF
2.   LSI
3. HDP





In [15]:
import os
import re
import operator
import matplotlib.pyplot as plt
import warnings
import gensim
import numpy as np
warnings.filterwarnings('ignore')

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel, Nmf
# from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from pprint import pprint

%matplotlib inline

In [40]:
# Noun extract and lemmatize function
def nouns(text):
    '''Given a string of text, tokenize the text 
    and pull out only the nouns.'''

    # create mask to isolate words that are nouns
    is_noun = lambda pos: pos[:2] == 'NN'

    # store function to split string of words 
    # into a list of words (tokens)
    tokenized = word_tokenize(text)

    # store function to lemmatize each word
    wordnet_lemmatizer = WordNetLemmatizer()

    # use list comprehension to lemmatize all words 
    # and create a list of all nouns
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
    for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 

    # remove words having single character
    all_nouns = [word for word in all_nouns if len(word)>1]
    
    #return list of nouns
    return all_nouns


# Create dataframe of only nouns from article text
data['nouns'] = data.clean_text.apply(nouns)

In [42]:
# remove articles which have 2 or less than 2 nouns in it
data = data[data.nouns.apply(len)>2]
data.shape

(2690, 6)

In [47]:
# create corpus and dictionary
train_texts = data['nouns'].tolist()
dictionary = Dictionary(train_texts)
corpus = [dictionary.doc2bow(text) for text in train_texts]

In [48]:
# NMF Model
nmfmodel = Nmf(corpus=corpus, num_topics=7, id2word=dictionary, kappa=0.1, eval_every=5)

nmftopics = nmfmodel.show_topics(formatted=False)

In [49]:
pprint(nmftopics)

[(0,
  [('test', 0.029089962164463894),
   ('ball', 0.01954552002665402),
   ('cricket', 0.019348289750149655),
   ('day', 0.018781774041575983),
   ('player', 0.01555715203402291),
   ('england', 0.013818693201815087),
   ('match', 0.013414897076203207),
   ('wicket', 0.012310249589552007),
   ('run', 0.011796597375073652),
   ('oil', 0.011067091521632424)]),
 (1,
  [('pakistan', 0.024583770424552686),
   ('wicket', 0.01527691388622596),
   ('government', 0.011237386129849224),
   ('minister', 0.010691469138516575),
   ('run', 0.010485425851710266),
   ('tax', 0.010127952337394358),
   ('ball', 0.009504461357664841),
   ('england', 0.009332671079829453),
   ('match', 0.00872760129003895),
   ('bank', 0.0071356248324574425)]),
 (2,
  [('year', 0.06742757722423043),
   ('percent', 0.02446241024203379),
   ('month', 0.0199995497972316),
   ('country', 0.01308434283517102),
   ('bank', 0.011179548233720683),
   ('dollar', 0.010072243897562106),
   ('sector', 0.008414276154293032),
   ('we

In [50]:
# LSI Model
lsimodel = LsiModel(corpus=corpus, num_topics=7, id2word=dictionary)

lsitopics = lsimodel.show_topics(formatted=False)

In [51]:
pprint(lsitopics)

[(0,
  [('percent', 0.4319514086819011),
   ('year', 0.37905087462964376),
   ('oil', 0.2663824623667386),
   ('market', 0.25987269135819885),
   ('price', 0.2344185000779664),
   ('week', 0.1511393575356322),
   ('month', 0.1486139500271097),
   ('rate', 0.12541454532146915),
   ('dollar', 0.123923674451992),
   ('barrel', 0.12328185399273175)]),
 (1,
  [('test', 0.32010181101752244),
   ('wicket', 0.26440844020917464),
   ('match', 0.2548688897703744),
   ('ball', 0.24260271449071202),
   ('england', 0.21638115310114286),
   ('team', 0.21397723393924709),
   ('cricket', 0.20075561435005726),
   ('run', 0.1898957284559217),
   ('pakistan', 0.1825129418612685),
   ('percent', -0.16908417028617706)]),
 (2,
  [('oil', -0.4767832953407647),
   ('price', -0.3176129975743934),
   ('percent', 0.29407046483958516),
   ('year', 0.24675137831408772),
   ('barrel', -0.24665537678410976),
   ('crude', -0.21228745204648175),
   ('tax', 0.17937932919586205),
   ('government', 0.14403490540952366),


In [52]:
# HDP Model
# An HDP model is fully unsupervised. It can also determine the ideal number of topics it needs through posterior inference.

hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)
hdptopics = hdpmodel.show_topics(formatted=False)
pprint(hdptopics)

[(0,
  [('year', 0.01405304514159855),
   ('percent', 0.010202530249196347),
   ('world', 0.007663094502861367),
   ('oil', 0.0069963846866198855),
   ('pakistan', 0.006932804955832594),
   ('market', 0.006498113092517903),
   ('test', 0.006476077567732091),
   ('price', 0.005935172502158944),
   ('day', 0.005902486124823098),
   ('match', 0.005843255305730751),
   ('team', 0.005750285958663328),
   ('country', 0.00554990781322948),
   ('month', 0.0055020186760097465),
   ('time', 0.0053301486475012365),
   ('england', 0.00481616034954262),
   ('cricket', 0.004742615439880164),
   ('week', 0.00471780170574257),
   ('wicket', 0.004238957950105355),
   ('ball', 0.004104523416785958),
   ('game', 0.0038014075768668513)]),
 (1,
  [('year', 0.013931624631219549),
   ('percent', 0.013828594497064245),
   ('oil', 0.008922375896242436),
   ('market', 0.008198467509684455),
   ('price', 0.007345565139354563),
   ('week', 0.005968485662965219),
   ('world', 0.0055894895562856785),
   ('day', 0.0

### LDA Model

In [53]:
ldamodel = LdaModel(corpus=corpus, num_topics=7, id2word=dictionary)
ldatopics = ldamodel.show_topics(formatted=False)
pprint(ldatopics)

[(0,
  [('world', 0.0125458315),
   ('game', 0.012419954),
   ('year', 0.010231936),
   ('country', 0.008844093),
   ('rio', 0.008311693),
   ('time', 0.008266243),
   ('pakistan', 0.008034132),
   ('percent', 0.006484553),
   ('minister', 0.006221845),
   ('olympic', 0.005988608)]),
 (1,
  [('year', 0.012987602),
   ('pakistan', 0.008080203),
   ('world', 0.008071232),
   ('england', 0.006272058),
   ('match', 0.005682035),
   ('market', 0.0054119704),
   ('china', 0.0053962185),
   ('country', 0.005386768),
   ('government', 0.0053772186),
   ('day', 0.0051648356)]),
 (2,
  [('oil', 0.026282182),
   ('percent', 0.022724178),
   ('price', 0.020755026),
   ('market', 0.01478991),
   ('year', 0.014590325),
   ('barrel', 0.010815166),
   ('crude', 0.010435443),
   ('production', 0.008696468),
   ('week', 0.008526165),
   ('month', 0.008432529)]),
 (3,
  [('year', 0.02255285),
   ('percent', 0.011282489),
   ('pakistan', 0.008063862),
   ('country', 0.007693108),
   ('month', 0.006807861)

In [54]:
#Finding out the optimal number of topics using topic coherence

def evaluate_graph(dictionary, corpus, texts, min_limit, max_limit):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    for num_topics in range(min_limit, max_limit):
        lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
        
    # Show graph
    x = range(min_limit, max_limit)
    plt.plot(x, c_v)
    plt.xlabel("num_topics")
    plt.ylabel("Coherence score")
    plt.legend(("c_v"), loc='best')
    plt.show()
    
    return lm_list, c_v

In [55]:
%%time
lmlist, c_v = evaluate_graph(dictionary=dictionary, corpus=corpus, texts=train_texts, min_limit=3, max_limit=12)

CPU times: user 37.9 s, sys: 392 ms, total: 38.3 s
Wall time: 38.6 s


In [56]:
# best model among optimal number of topics 
#kindly choose the index position for best model from above graph 
idx = 2
lda_best_topics = lmlist[idx].show_topics(formatted=False)

In [59]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

In [60]:
# visualizing the model which had best coherence score from above #topics evaluation

# trying model having 5 number of topics as it gave best coherence score in above chart
pyLDAvis.gensim_models.prepare(lmlist[idx], corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.106770 -0.049086       1        1  24.763901
0     -0.119561 -0.061519       2        1  23.792361
2      0.075627  0.012678       3        1  18.540366
1     -0.022687  0.049175       4        1  18.295525
4     -0.040149  0.048753       5        1  14.607846, topic_info=         Term         Freq        Total Category  logprob  loglift
3517     test  1820.000000  1820.000000  Default  30.0000  30.0000
118       oil  1554.000000  1554.000000  Default  29.0000  29.0000
4478  england  1234.000000  1234.000000  Default  28.0000  28.0000
121   percent  2001.000000  2001.000000  Default  27.0000  27.0000
6963   wicket  1144.000000  1144.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
157      time   169.936514  1432.950247   Topic5  -5.2912  -0.2085
81     friday   109.975959   463.361854   Topic5  -5.7263   0.4854
1176   minute   106.306486   433.134889   Topic5  -5.7603   0.5189
44      china   113.173233   662.411290   Topic5  -5.6977   0.1566
2019     game   109.831107  1009.675397   Topic5  -5.7277  -0.2948

[422 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
3174       1  0.904865       agent
3174       2  0.034802       agent
3174       3  0.034802       agent
3174       4  0.034802       agent
3174       5  0.034802       agent
...      ...       ...         ...
7807       5  0.052125      younis
10983      5  0.835038      zagreb
1211       5  0.742558  zuckerberg
9138       3  0.846164      zverev
14579      2  0.949987      china

[1170 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2, 5])

In [48]:
# One of the problem with LDA is that if we train it on a large number of topics, the topics get "lost" among the numbers. 
# Let us try to dig out the best topics from the best LDA model we can produce. 
# The function below can be used to control the quality of the LDA model we produce.

In [62]:
threshold = 0.6 # you can change to a higher number but it will take more time

def ret_top_model():
    """
    Since LDAmodel is a probabilistic model, it comes up different topics each time we run it. To control the
    quality of the topic model we produce, we can see what the interpretability of the best topic is and keep
    evaluating the topic model until this threshold is crossed. 
    
    Returns:
    -------
    lm: Final evaluated topic model
    top_topics: ranked topics in decreasing order. List of tuples
    """
    top_topics = [(0, 0)]
    while top_topics[0][1] < threshold:
        lm = LdaModel(corpus=corpus, id2word=dictionary)
        coherence_values = {}
        for n, topic in lm.show_topics(num_topics=-1, formatted=False):
            topic = [word for word, _ in topic]
            cm = CoherenceModel(topics=[topic], texts=train_texts, dictionary=dictionary, window_size=10)
            coherence_values[n] = cm.get_coherence()
        top_topics = sorted(coherence_values.items(), key=operator.itemgetter(1), reverse=True)
    return lm, top_topics

In [63]:
%%time
lm, top_topics = ret_top_model()

CPU times: user 3min 23s, sys: 6.77 s, total: 3min 30s
Wall time: 3min 38s


In [64]:
print(top_topics[:5])

[(56, 0.7455696526646152), (92, 0.7042349580612363), (98, 0.7002422788749558), (3, 0.6745855797433052), (21, 0.6616614795157859)]


In [65]:
pprint([lm.show_topic(topicid) for topicid, c_v in top_topics[:5]])

[[('oil', 0.06923694),
  ('price', 0.033335857),
  ('barrel', 0.029803952),
  ('crude', 0.027133817),
  ('production', 0.023823606),
  ('output', 0.022592625),
  ('market', 0.020402435),
  ('cut', 0.015988143),
  ('opec', 0.01579512),
  ('year', 0.014898591)],
 [('bpd', 0.023872005),
  ('price', 0.020445203),
  ('oil', 0.020101983),
  ('production', 0.018376436),
  ('market', 0.017560938),
  ('year', 0.015070894),
  ('barrel', 0.014739988),
  ('output', 0.013764119),
  ('opec', 0.012476302),
  ('meeting', 0.011319977)],
 [('wicket', 0.04357664),
  ('ball', 0.036811475),
  ('run', 0.030498015),
  ('match', 0.025470842),
  ('england', 0.019563377),
  ('spinner', 0.019081127),
  ('inning', 0.019067325),
  ('over', 0.018755276),
  ('pakistan', 0.016798424),
  ('bowler', 0.015425294)],
 [('test', 0.0580004),
  ('england', 0.026184352),
  ('day', 0.018465351),
  ('series', 0.018066341),
  ('inning', 0.0153382765),
  ('year', 0.014821558),
  ('wicket', 0.01286417),
  ('cook', 0.011980765),
  

### Evaluating all topics model

In [68]:
#NMF
nmftopics = [[word for word, prob in topic] for topicid, topic in nmftopics]

#LSI
lsitopics = [[word for word, prob in topic] for topicid, topic in lsitopics]

#HDP
hdptopics = [[word for word, prob in topic] for topicid, topic in hdptopics]

#LDA
ldatopics = [[word for word, prob in topic] for topicid, topic in ldatopics]

# best LDA from optimal number of topics evaluation using topic coherence
lda_best_topics = [[word for word, prob in topic] for topicid, topic in lda_best_topics]

# top LDA model based on user defined topic coherence threshold (we gave 0.6)
lda_top_topics = [[word for word, prob in topic] for topicid, topic in lda_top_topics]

In [69]:
nmf_coherence = CoherenceModel(topics=nmftopics[:7], texts=train_texts, dictionary=dictionary, window_size=10).get_coherence()

lsi_coherence = CoherenceModel(topics=lsitopics[:7], texts=train_texts, dictionary=dictionary, window_size=10).get_coherence()

hdp_coherence = CoherenceModel(topics=hdptopics[:7], texts=train_texts, dictionary=dictionary, window_size=10).get_coherence()

lda_coherence = CoherenceModel(topics=ldatopics, texts=train_texts, dictionary=dictionary, window_size=10).get_coherence()

lm_best_coherence = CoherenceModel(topics=lda_best_topics, texts=train_texts, dictionary=dictionary, window_size=10).get_coherence()

lda_top_coherence = CoherenceModel(topics=lda_top_topics[:7], texts=train_texts, dictionary=dictionary, window_size=10).get_coherence()

In [70]:
def evaluate_bar_graph(coherences, indices):
    """
    Function to plot bar graph.
    
    coherences: list of coherence values
    indices: Indices to be used to mark bars. Length of this and coherences should be equal.
    """
    assert len(coherences) == len(indices)
    n = len(coherences)
    x = np.arange(n)
    plt.bar(x, coherences, width=0.2, tick_label=indices, align='center')
    plt.xlabel('Models')
    plt.ylabel('Coherence Value')

In [73]:
evaluate_bar_graph([nmf_coherence, lsi_coherence, hdp_coherence, lda_coherence, lm_best_coherence, lda_top_coherence],
                   ['NMF', 'LSI', 'HDP', 'LDA', 'LDA_Best', 'LDA_Top'])

### NMF Topic Modeling using scikit learn (you can ignore)

In [ ]:
# Noun extract and lemmatize function
def nouns(text):
    '''Given a string of text, tokenize the text 
    and pull out only the nouns.'''
    # create mask to isolate words that are nouns
    is_noun = lambda pos: pos[:2] == 'NN'
    # store function to split string of words 
    # into a list of words (tokens)
    tokenized = word_tokenize(text)
    # store function to lemmatize each word
    wordnet_lemmatizer = WordNetLemmatizer()
    # use list comprehension to lemmatize all words 
    # and create a list of all nouns
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
    for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    
    #return string of joined list of nouns
    return ' '.join(all_nouns)


# Create dataframe of only nouns from articles text
data['nouns'] = data.clean_text.apply(nouns)

In [ ]:
# Add additional stop words since we are recreating the document-term matrix
# stop_noun = ["strong", 'DUBAI', 'WASHINGTON', 'BEIJING']
# stop_words_noun_agg = text.ENGLISH_STOP_WORDS.union(stop_noun)
stopwords = stopwords.words('english')

# Create a document-term matrix with only nouns
# Store TF-IDF Vectorizer
tv_noun = TfidfVectorizer(stop_words=stopwords, ngram_range = (1,2), max_df = .8, min_df = .01)

# Fit and Transform speech noun text to a TF-IDF Doc-Term Matrix
data_tv_noun = tv_noun.fit_transform(data.nouns)

In [ ]:
# Create data-frame of Doc-Term Matrix with nouns as column names
data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names())

# Visually inspect Document Term Matrix
data_dtm_noun.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


aaron   ab  ab villiers  ability  absence  access  account  achievement  \
0    0.0  0.0          0.0      0.0      0.0     0.0      0.0          0.0   
1    0.0  0.0          0.0      0.0      0.0     0.0      0.0          0.0   
2    0.0  0.0          0.0      0.0      0.0     0.0      0.0          0.0   
3    0.0  0.0          0.0      0.0      0.0     0.0      0.0          0.0   
4    0.0  0.0          0.0      0.0      0.0     0.0      0.0          0.0   

   act    action  ...       yen  yen york  yield      york  younis  yuan  \
0  0.0  0.000000  ...  0.000000       0.0    0.0  0.000000     0.0   0.0   
1  0.0  0.000000  ...  0.055147       0.0    0.0  0.000000     0.0   0.0   
2  0.0  0.000000  ...  0.000000       0.0    0.0  0.000000     0.0   0.0   
3  0.0  0.000000  ...  0.000000       0.0    0.0  0.123566     0.0   0.0   
4  0.0  0.091382  ...  0.000000       0.0    0.0  0.000000     0.0   0.0   

   zalmi  zealand  zimbabwe  zone  
0    0.0      0.0       0.0   0.0  
1    0.0      0.0       0.0   0.0  
2    0.0      0.0       0.0   0.0  
3    0.0      0.0       0.0   0.0  
4    0.0      0.0       0.0   0.0  

[5 rows x 1380 columns]

In [ ]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
  '''
  Given an NMF model, feature_names, and number of top words, 
  print topic number and its top feature names, up to specified number of top words.
  '''
  # iterate through topics in topic-term matrix, 'H' aka
  # model.components_
  for ix, topic in enumerate(model.components_):
      #print topic, topic number, and top words
      if not topic_names or not topic_names[ix]:
          print("\nTopic ", ix)
      else:
          print("\nTopic: '",topic_names[ix],"'")
      print(", ".join([feature_names[i] \
            for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [ ]:
nmf_model = NMF(7) # change the number of topics by entering the number of your choice

# Learn an NMF model for given Document Term Matrix 'V' 
# Extract the document-topic matrix 'W'
doc_topic = nmf_model.fit_transform(data_dtm_noun)

# Extract top words from the topic-term matrix 'H' 
display_topics(nmf_model, tv_noun.get_feature_names_out(), 7) #change the number if you wish to see more terms within each topic

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,



Topic  0
game, world, time, minute, goal, year, champion

Topic  1
test, england, series, day, cook, inning, lanka

Topic  2
oil, price, barrel, crude, market, production, oil price

Topic  3
percent, market, rate, dollar, index, stock, bank

Topic  4
minister, government, pakistan, tax, country, power, finance

Topic  5
wicket, ball, run, four, match, six, bowler

Topic  6
cricket, team, pakistan, pcb, india, board, pakistan cricket
